In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import mne
from data_processing import compute_lags, apply_lags

In [3]:
array = np.random.randint(low=1, high=20, size=70)

lags = [3,5,-2,7,-5,4,-1,9,-8,8]
E=10
central_array_cropped = array[E:len(array)-E]
len(central_array_cropped), central_array_cropped

(50,
 array([ 2, 17, 17,  3, 10,  8,  1,  2,  6, 19,  8,  2, 19, 19,  4, 16,  4,
         5,  3,  8,  1, 18, 18,  9, 10, 18,  6, 15,  4, 13, 18,  6,  3,  3,
         4, 17, 16, 13, 18,  9,  3, 15, 15, 19, 10,  1,  5, 14,  2, 18]))

In [4]:
array

array([13, 18, 14, 16, 13,  7,  7,  2,  8, 13,  2, 17, 17,  3, 10,  8,  1,
        2,  6, 19,  8,  2, 19, 19,  4, 16,  4,  5,  3,  8,  1, 18, 18,  9,
       10, 18,  6, 15,  4, 13, 18,  6,  3,  3,  4, 17, 16, 13, 18,  9,  3,
       15, 15, 19, 10,  1,  5, 14,  2, 18, 14,  6, 16, 11, 15, 17, 13,  8,
       15,  5])

In [5]:
l=3
array[E+l:len(array)-E+l]

array([ 3, 10,  8,  1,  2,  6, 19,  8,  2, 19, 19,  4, 16,  4,  5,  3,  8,
        1, 18, 18,  9, 10, 18,  6, 15,  4, 13, 18,  6,  3,  3,  4, 17, 16,
       13, 18,  9,  3, 15, 15, 19, 10,  1,  5, 14,  2, 18, 14,  6, 16])

In [6]:
lagged_cropped_arrays = [array[E+l:len(array)-E+l] for l in lags]

#stuff the borders with random values
lagged_arrays =[np.concatenate([np.random.randint(low=1,high=20, size=E), lga, np.random.randint(low=1,high=20, size=E)]) for lga in lagged_cropped_arrays]

#add channel_dim
lagged_arrays = [np.expand_dims(la, axis=0) for la in lagged_arrays]

lagged_arrays = np.stack(lagged_arrays)
lagged_arrays.shape

(10, 1, 70)

In [7]:
central_arrays = np.repeat(np.expand_dims([array], axis=0),repeats=40, axis=0)
central_arrays.shape #n_epochs,n_channels,n_times

(40, 1, 70)

In [8]:
#concatenate centralized and lagged arrays
e_ = np.concatenate([central_arrays, lagged_arrays])
e_.shape, e_

((50, 1, 70),
 array([[[13, 18, 14, ...,  8, 15,  5]],
 
        [[13, 18, 14, ...,  8, 15,  5]],
 
        [[13, 18, 14, ...,  8, 15,  5]],
 
        ...,
 
        [[19,  4,  4, ..., 12, 15,  8]],
 
        [[ 9,  2, 13, ...,  4, 16, 14]],
 
        [[ 1, 13, 19, ...,  2, 19,  1]]]))

In [9]:
info_ = mne.create_info(ch_names=1,sfreq = 50)
epochs_ = mne.EpochsArray(e_,info_)

Not setting metadata
50 matching events found
No baseline correction applied
0 projection items activated


In [26]:
# lags_list, cond_hist
lags_list, cond_hist = compute_lags(epochs_, similarity="covariance", criteria_sim="greatest_local_max", E=E)         

E:  10
Iteration num:  0
cavalo


In [27]:
lags_list

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 -3,
 -5,
 2,
 -7,
 5,
 -4,
 1,
 -9,
 8,
 -8]

In [28]:
lags_list[-10:], lags

([-3, -5, 2, -7, 5, -4, 1, -9, 8, -8], [3, 5, -2, 7, -5, 4, -1, 9, -8, 8])

In [29]:
lagged_epochs_ = apply_lags(epochs_, E, [l for l in lags_list])

In [30]:
lagged_epochs_.get_data()

array([[[ 2., 17., 17., ..., 14.,  2., 18.]],

       [[ 2., 17., 17., ..., 14.,  2., 18.]],

       [[ 2., 17., 17., ..., 14.,  2., 18.]],

       ...,

       [[ 4.,  4., 17., ..., 14.,  2., 18.]],

       [[ 2., 17., 17., ...,  8.,  2.,  4.]],

       [[19., 14., 15., ..., 14.,  2., 18.]]])

In [49]:
lagged_epochs_.get_data()[-1]

array([[ 5.,  7.,  4.,  9.,  1., 18., 10., 12., 17., 15.,  7., 11., 16.,
         2.,  9., 12., 10., 13.,  3., 15.,  3., 13., 19.,  2.,  1.,  4.,
        18., 10., 11., 12., 11., 17., 11.,  2., 17.,  3., 17., 15.,  8.,
        18.,  4.,  8., 18.,  2., 18., 10., 18.,  4.,  4.,  9.]])

In [50]:
lagged_epochs_.get_data()[-1]

array([[ 5.,  7.,  4.,  9.,  1., 18., 10., 12., 17., 15.,  7., 11., 16.,
         2.,  9., 12., 10., 13.,  3., 15.,  3., 13., 19.,  2.,  1.,  4.,
        18., 10., 11., 12., 11., 17., 11.,  2., 17.,  3., 17., 15.,  8.,
        18.,  4.,  8., 18.,  2., 18., 10., 18.,  4.,  4.,  9.]])

In [51]:
lagged_arrays[-1]

array([[ 2, 15, 16, 13,  2, 14, 17, 10, 18, 11, 15, 19, 12, 10, 18,  5,
         9, 19,  5,  7,  4,  9,  1, 18, 10, 12, 17, 15,  7, 11, 16,  2,
         9, 12, 10, 13,  3, 15,  3, 13, 19,  2,  1,  4, 18, 10, 11, 12,
        11, 17, 11,  2, 17,  3, 17, 15,  8, 18,  4,  8, 18,  2, 18, 10,
        18,  4,  4,  9,  8,  3]])

In [52]:
lagged_epochs_.get_data()[0]

array([[19., 10., 10., 18., 11.,  9.,  2.,  8., 15., 19., 12., 10., 18.,
         5.,  9., 19.,  5.,  7.,  4.,  9.,  1., 18., 10., 12., 17., 15.,
         7., 11., 16.,  2.,  9., 12., 10., 13.,  3., 15.,  3., 13., 19.,
         2.,  1.,  4., 18., 10., 11., 12., 11., 17., 11.,  2.]])

In [28]:
len(lagged_epochs_.get_data()[0].ravel())

50

In [32]:
len(e_[0].ravel())
e_[0]

array([[11,  8, 12,  9,  8, 17,  7, 10, 16, 13,  5, 11, 15, 15, 13,  9,
        17,  7, 14,  2, 16,  5, 15,  5, 14,  8, 11,  5,  8, 15,  2,  4,
        10,  1, 19, 13,  6, 15, 19,  1, 13, 16,  1,  8,  4,  5, 15,  9,
        10, 19,  7,  8,  9, 14,  5,  5, 13, 19,  8,  8,  9,  2, 15,  7,
        17, 15, 10,  4,  7, 14]])

In [33]:
central_array_cropped

array([ 5, 11, 15, 15, 13,  9, 17,  7, 14,  2, 16,  5, 15,  5, 14,  8, 11,
        5,  8, 15,  2,  4, 10,  1, 19, 13,  6, 15, 19,  1, 13, 16,  1,  8,
        4,  5, 15,  9, 10, 19,  7,  8,  9, 14,  5,  5, 13, 19,  8,  8])